# Walkthrough for backend

You will need a firestore database authentication file aka a Firebase Admin SDK.
Upload this anywhere in the src folder and copy the relative path to the file to SDK_path as shown below

In [1]:
#Load SDK
import firebase_admin
from firebase_admin import credentials, firestore
from APIs.backend import *
###############################################################################################
SDK_path = "path/to/auth.json" #Update this!
###############################################################################################
try:
    cred = credentials.Certificate(SDK_path)
    firebase_admin.initialize_app(cred)
except:
    print("Already loaded")
db = firestore.client()

Within your firestore, you will need to create collections and initiate them with the correct file structure. For playlists structure, see the description of the backend.py's add_playlists function. 

In our examples, we have 3 collections:
- test: A collection of data containing only the feature, 'playlist_name'
- playlist_selected: A collection of playlists following the structure described in backend.py's add_playlists
- references: A collected of tracks and their features. 

To read and write, use add_playlists, clear_collection, get_playlists, get_track_ids, or load_reference_data

### General: Add playlist to collection or clear collection:
The collection that we add to must have the correct structure. In this example, each collection has only 1 feature (the playlist_name) but in the actual setup, we use a nested playlist+song_ids structure. See backend.py for detailed instructions.

In [2]:
# Add a playlist to an existing collection:
test = [{'playlist_name':'1'},{'playlist_name':'c'}] #Playlist to add. Simple case, but can be as complex as needed
collection = 'test' #Name of collection to add to
add_playlists(test, collection, auth=SDK_path)

Authentication already loaded


Let's view what's in our database.


In [3]:
#Access and view the current tracks
data = get_playlists(collection,auth=default_auth_path)
data

Authentication already loaded


[{'playlist_name': '1'}, {'playlist_name': 'c'}]

In [4]:
#Clear collection: Lets clear the data we just added
collection = 'test' #Name of collection to add to
clear_collection(collection)

Authentication already loaded
Deleting Playlist: 1
Deleting Playlist: c


### Specific to this project:
This project has a predefined dictionary storage system for each playlist. This is defined in backend.py.
The following functions depend on this structure and will fail otherwise:

get_track_ids, or load_reference_data

- The collection, 'playlist_selected' is a collection of playlist objects as previously described.
- The collection, 'reference' is a set of saved track features. This is the set choose from when creating new playlist

In [16]:
#get_track_ids

# We first get the playlists returned by get_playlists. Then we feed that into get_track_ids.
collection = 'playlists_selected'
playlists = get_playlists(collection)
print('Playlists \n', playlists[0]) # Lets view the first playlist in playlists

# Lets get all the ids of songs in these playlist. Repeats are not included
ids = get_track_ids(playlists)
print("Song ids:{}".format(ids ))


Authentication already loaded
Playlists 
 {'playlist_id': '1', 'playlist_name': '1', 'playlist_img_url': '1', 'playlist_tracks_url': '1', 'playlist_url': '1', 'playlist_tracks': [{'track_url': 'd', 'track_id': '5', 'track_artist': 'a', 'track_image': 'c', 'track_name': 'b'}, {'track_image': 'c2', 'track_id': '10', 'track_artist': 'a2', 'track_url': 'd2', 'track_name': 'b2'}]}
Song ids:['5', '10']


In [12]:
#load_reference_data
collection = 'references' 
num_tracks = 10 #how many tracks we get
sort_by = None #sort data before accessing. Note this means we get the top points by this metric. ex sort_by = 'tempo'
auth = SDK_path
selected_features = ['tempo'] #The features we want to be returned
df = load_reference_data(collection,num_tracks,sort_by,auth, selected_features)
df.head()

Authentication already loaded


,tempo
0,113.756
0,178.258
0,93.973
0,95.902
0,86.867


# Walkthrough of preprocess:
Below, replace the client_id and client_secret with your spotify developer info

In [13]:
from preprocess import *
ids = ["31MkGwj0SWgZaTvEt5Hgxv"] # Example song id
client_id = "" # REPLACE THIS WITH YOUR OWN
client_secret = "" #REPLACE WITH YOUR OWN
df = get_features_from_ids(ids, client_id, client_secret)
df.head()

c:\D_drive\Columbia 2022\Big data\Project\spotify-big-data-project\src\preprocess.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tf_df = tf_df.append(feat, ignore_index=True)


,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,0.0252,0.386,271520,0.594,0.000003,1,0.0873,-6.186,1,0.0566,113.756,0.187
